In [118]:
pip install torch

In [119]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

In [120]:
news_df = pd.read_excel('/content/lg_final.xlsx')
news_df.head()

,Unnamed: 0,predict_date,article_text,oil_news,엘지종가,엘지타겟
0,0,2022-01-02,[아시아경제 송화정 기자]외국인이 국내 증시에서 3주 연속 매수세를 나타냈다.\n\...,[충청투데이 심형식 기자] 코로나19(이하 코로나)가 2년가까이 지속되면서 대면서비...,NaN,NaN
1,1,2022-01-03,사진 확대 자본시장의 상징이자 상승장을 기원하는 황소상. 이 황소상은 지난 1996...,▲ 2021년 12월 충청지역 소비자물가동향\n\n충북과 충남의 물가 상승률이 4%...,618000.0,0.0
2,2,2022-01-04,"viewer 4일 코스피 지수는 전장보다 3.20포인트(0.11%) 오른 2,991...","\n\n서울 시내 주유소 모습.\n\n\n\n\n\n[ 저작권자 ⓒ디지털타임스, 무...",644000.0,1.0
3,3,2022-01-05,반면 같은 기간 동학개미 순매수 1위였던 삼성전자(보통주)의 수익률은 -3.2%에 ...,"< 저작권자 ⓒ 서울경제, 무단 전재 및 재배포 금지 >\n\n미국 뉴욕증시의 주요...",664000.0,1.0
4,4,2022-01-06,[사진=게티이미지뱅크]\n\nLG화학은 미국 면역항암제 개발 파트너사 '큐바이오파마...,美·남미 등 수입처 다변화\n\n안정적 물량 확보는 어려워\n\n\n\n코로나19 ...,690000.0,1.0


In [121]:
model_name = "beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [122]:
texts = news_df['article_text'].tolist()
oil_news = news_df['oil_news'].tolist()

In [123]:
encoded_inputs = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')

In [124]:
with torch.no_grad():
    outputs = model(**encoded_inputs)

In [125]:
embeddings = outputs.last_hidden_state[:, 0, :].numpy()

In [126]:
news_df['kcbert_embedding'] = embeddings.tolist()

In [127]:
encoded_inputs_oil = tokenizer(oil_news, truncation=True, padding=True, return_tensors='pt')
with torch.no_grad():
    outputs_oil = model(**encoded_inputs_oil)
embeddings_oil = outputs_oil.last_hidden_state[:, 0, :].numpy()
news_df['kcbert_embedding_oil'] = embeddings_oil.tolist()

In [128]:
news_df = news_df.drop(['Unnamed: 0'],axis=1)

In [129]:
news_df = news_df.dropna(axis=0)

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:


# 훈련 데이터와 레이블 준비
X = news_df[['kcbert_embedding', 'kcbert_embedding_oil']]  # 특성 데이터 (임베딩 벡터)
y = news_df['엘지타겟']  # 레이블 (주식 상승/하락)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 훈련 세트와 테스트 세트로 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear')  # 선형 SVM 모델
svm_model.fit(X_train, y_train)

# 테스트 세트 예측
y_pred = svm_model.predict(X_test)

# 분류 모델 성능 평가
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 성능 지표 출력
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
news_df['kcbert_embedding_oil']